In [ ]:
import cv2
import numpy as np
from openpose import pyopenpose as op
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, mean_absolute_error, mean_squared_error, r2_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración de OpenPose
params = {
    "model_folder": "openpose/models/",
    "net_resolution": "-1x368"
}
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

def extract_keypoints_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        datum = op.Datum()
        datum.cvInputData = frame
        opWrapper.emplaceAndPop([datum])
        keypoints = datum.poseKeypoints
        if keypoints is not None and len(keypoints) > 0:
            keypoints_list.append(keypoints[0])
    cap.release()
    return keypoints_list

def preprocess_keypoints(keypoints):
    keypoints = np.array(keypoints)
    max_val = np.max(keypoints, axis=(0, 1))
    min_val = np.min(keypoints, axis=(0, 1))
    normalized_keypoints = (keypoints - min_val) / (max_val - min_val)
    flattened_keypoints = normalized_keypoints.reshape(keypoints.shape[0], -1)
    return flattened_keypoints

def create_dataset(video_dir):
    data = []
    labels = []
    exercise_labels = {"sentadilla": 0, "dominadas": 1, "elevaciones_laterales": 2, "peso_muerto": 3, "plancha": 4, "press_banca": 5, "press_militar": 6}
    for exercise in exercise_labels.keys():
        exercise_dir = os.path.join(video_dir, exercise)
        for video_file in os.listdir(exercise_dir):
            video_path = os.path.join(exercise_dir, video_file)
            keypoints = extract_keypoints_from_video(video_path)
            preprocessed_keypoints = preprocess_keypoints(keypoints)
            for kp in preprocessed_keypoints:
                data.append(kp)
                labels.append(exercise_labels[exercise])
    return np.array(data), np.array(labels)

# Crear el dataset
video_dir = "videos"
X, y = create_dataset(video_dir)

# Convertir las etiquetas a one-hot encoding
y = to_categorical(y)

# Dividir el dataset en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir y compilar el modelo
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.5),
    Conv1D(128, 3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Realizar predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Matriz de Confusión
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=exercise_labels.keys(), yticklabels=exercise_labels.keys())
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Calcular MAE, MSE, RMSE, R2, y Precisión
mae = mean_absolute_error(y_true_classes, y_pred_classes)
mse = mean_squared_error(y_true_classes, y_pred_classes)
rmse = np.sqrt(mse)
r2 = r2_score(y_true_classes, y_pred_classes)
precision = accuracy_score(y_true_classes, y_pred_classes)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')
print(f'Precision: {precision}')
